In [1544]:
import pandas as pd
import requests
import urllib
from bs4 import BeautifulSoup
from datetime import datetime


***
#### I.  Part
Определите наиболее удобный способ программно получать список поданных за день форм,
автоматизируйте процесс загрузки данных и подготовки их для анализа.
***
Answer: An EDGAR database store daily filled filings in location /daily-index/, which will publish at the end of the day between 10:00 - 11:00PM. So we"ll parse link below with dynamic link and get that date what we need.
As an example I get and prepared Balance Sheet from quaterly report of HERMAN MILLER 

In [1545]:
# year and quater
year='2021'
quater='1'

# date format: yymmdd or '20210430'
date='20210104'

# it returns yymmdd format
# date = str(datetime.now().date()).replace('-','') 

# it returns us link to filings
def get_url(year, quater, date):
    url ='https://www.sec.gov/Archives/edgar/daily-index/%s/QTR%s/master.%s.idx' % (year,quater,date)

    return url

In [1704]:
# define a url, in this case I'll just pass the get_url function
file_url = get_url(year,quater,date)

# request that new content, this will not be a JSON STRUCTURE!
content = requests.get(file_url).content

# we're writing the content to a file, so we don't need to request it again.
with open('master_20210104.txt', 'wb') as f:
     f.write(content)
        
# opening dowloaded file
with open('master_20210104.txt','rb') as f:
     byte_data = f.read()

# we have a byte stream that needs to be decoded and then split by double spaces.
data = byte_data.decode("utf-8").split('  ')

# should remove the headers, so look for the end of the header and grab it's index
for index, item in enumerate(data):
    if "ftp://ftp.sec.gov/edgar/" in item:
        start_ind = index

# define a new dataset without the header info.
data_format = data[start_ind + 1:]

master_data = []

for index, item in enumerate(data_format):
    
    if index == 0:
        clean_item_data = item.replace('\n','|').split('|')
        clean_item_data = clean_item_data[8:]
    else:
        clean_item_data = item.replace('\n','|').split('|')
        
    for index, row in enumerate(clean_item_data):
        
        if '.txt' in row:
            mini_list = clean_item_data[(index - 4): index + 1]
            
            if len(mini_list) != 0:
                mini_list[4] = "https://www.sec.gov/Archives/" + mini_list[4]
                master_data.append(mini_list) 


In [1705]:
# our data collecting into DataFrame
df = pd.DataFrame(master_data, columns=['CIK','Company Name', 'FormType','Date', 'Source'])

# Date column converting into datetime format
df['Date']=df['Date'].apply(pd.to_datetime, format='%Y-%m-%d')

# replacing .txt format to html
df['Source'] = df['Source'].apply(lambda x: x.replace(".txt", "-index.html"))
df

,CIK,Company Name,FormType,Date,Source
0,1232579,SYRVALIN KRISTINE C,3,2021-01-04,https://www.sec.gov/Archives/edgar/data/123257...
1,1232582,ASHFORD HOSPITALITY TRUST INC,424B3,2021-01-04,https://www.sec.gov/Archives/edgar/data/123258...
2,1232582,ASHFORD HOSPITALITY TRUST INC,8-K,2021-01-04,https://www.sec.gov/Archives/edgar/data/123258...
3,1233152,CALHOUN DAVID L,4,2021-01-04,https://www.sec.gov/Archives/edgar/data/123315...
4,1233167,NEAL MICHAEL A,4,2021-01-04,https://www.sec.gov/Archives/edgar/data/123316...
...,...,...,...,...,...
4484,9631,BANK OF NOVA SCOTIA,FWP,2021-01-04,https://www.sec.gov/Archives/edgar/data/9631/0...
4485,9631,BANK OF NOVA SCOTIA,FWP,2021-01-04,https://www.sec.gov/Archives/edgar/data/9631/0...
4486,97216,TEREX CORP,3,2021-01-04,https://www.sec.gov/Archives/edgar/data/97216/...
4487,97517,TEXAS PACIFIC LAND TRUST,4,2021-01-04,https://www.sec.gov/Archives/edgar/data/97517/...


In [1706]:
# giving to function CIK number of Company
# we will see type of Forms and source index which is first column
def get_source(CIK):
    return df[df['CIK']==CIK]

get_source('66382')

,CIK,Company Name,FormType,Date,Source
3629,66382,HERMAN MILLER INC,10-Q,2021-01-04,https://www.sec.gov/Archives/edgar/data/66382/...


In [1707]:
# put the source index and form type from table above
# this function returns source to report

def report_source(source_index, form_type):
    #reading source and filling NaN value as an empty 
    document_index = pd.read_html(df['Source'][source_index])
    document_index = document_index[0].fillna('')
    
    # here we're getting access to each report
    document_name = document_index[document_index['Type'].str.contains(form_type)]
    document_name = document_name['Document'].str.split(' ')
    document_name = document_name[0][0]
    
    report_f = df['Source'][source_index].replace('-','').replace('index.html','')
    
    return report_f + '/' + document_name

In [1708]:
print(report_source(3629, '10-Q'))

https://www.sec.gov/Archives/edgar/data/66382/000006638221000003/mlhr-20201128.htm


In [1709]:
# This exaple of preparing 10-Q report
# reading function returns source
# it might right here some 403 forbidden problem should run several times

url_fin = report_source(3629, '10-Q')
balance_sheet = pd.read_html(url_fin)

In [1710]:
# finding balancesheet
balance_sheet = balance_sheet[7][[0,4,10]]

In [1711]:
#deleting unnecessary data
balance_sheet.columns=['Accounts','28/11/2020','30/05/2020']
balance_sheet = balance_sheet.dropna()
balance_sheet = balance_sheet.drop([2,23])

In [1712]:
balance_sheet=balance_sheet.reset_index(drop=True)
balance_sheet.head(10)

,Accounts,28/11/2020,30/05/2020
0,Cash and cash equivalents,377.9,454.0
1,Short-term investments,7.2,7.0
2,"Accounts receivable, net of allowances of $5.2...",193.0,180.0
3,Unbilled accounts receivable,33.0,19.5
4,"Inventories, net",191.0,197.3
5,Prepaid expenses,26.8,43.3
6,Other current assets,9.0,16.0
7,Total current assets,837.9,917.1
8,"Property and equipment, at cost",1124.0,1111.3
9,Less — accumulated depreciation,(798.1),(780.5)


In [1713]:
# cleaning and preparing data
balance_sheet['28/11/2020'] = balance_sheet['28/11/2020'].map(lambda x: x.replace('(', '-').replace(')', ''))
balance_sheet['30/05/2020'] = balance_sheet['30/05/2020'].map(lambda x: x.replace('(', '-').replace(')', ''))
balance_sheet['28/11/2020'] = balance_sheet['28/11/2020'].map(lambda x: x.replace('—', '0'))
balance_sheet['30/05/2020'] = balance_sheet['30/05/2020'].map(lambda x: x.replace('—', '0'))


In [1717]:
# if we want we can store balance sheet to csv
balance_sheet[['28/11/2020', '30/05/2020']] = balance_sheet[['28/11/2020', '30/05/2020']].astype(float)
balance_sheet.head(10)

,Accounts,28/11/2020,30/05/2020
0,Cash and cash equivalents,377.9,454.0
1,Short-term investments,7.2,7.0
2,"Accounts receivable, net of allowances of $5.2...",193.0,180.0
3,Unbilled accounts receivable,33.0,19.5
4,"Inventories, net",191.0,197.3
5,Prepaid expenses,26.8,43.3
6,Other current assets,9.0,16.0
7,Total current assets,837.9,917.1
8,"Property and equipment, at cost",1124.0,1111.3
9,Less — accumulated depreciation,-798.1,-780.5


In [1715]:
# here exaple some calculations
total_current_assets = balance_sheet['28/11/2020'][0:7].sum()
print('Total current assets:',total_current_assets)
print('Total current liabilities:', balance_sheet['28/11/2020'][23])
print('Current Ratio:', total_current_assets/balance_sheet['28/11/2020'][23])

Total current assets: 837.8999999999999
Total current liabilities: 507.5
Current Ratio: 1.6510344827586205


***
Because of daily filings will publish at the end of the day, we can use atom output format to getting live filings

In [1564]:
# control list
control = []
new_data = {}

In [1724]:
# Live time getting filings
# This parser let us know last 40 filing that filled in each time and if it is not in our dataframe it show us new filings
# so then we can find by CIK or Source and pars it for quick analyze
import feedparser
feed = feedparser.parse('https://www.sec.gov/cgi-bin/browse-edgar?action=getcurrent&CIK=&type=&company=&dateb=&owner=include&start=0&count=40&output=atom')

for entry in feed.entries:
    new_data.setdefault('Title', [])
    new_data.setdefault('Source', [])
    if entry.title not in control:
        control.append(entry.title)
        new_data['Title'].append(entry.title)
        new_data['Source'].append(entry.link)
        print(entry.title)

In [1723]:
df = pd.DataFrame(new_data)
df[::-1]

,Title,Source
176,4 - Thimsen John (0001842163) (Reporting),https://www.sec.gov/Archives/edgar/data/184216...
175,3 - Pavia Juan Carlos (0001860670) (Reporting),https://www.sec.gov/Archives/edgar/data/186067...
174,4 - NOWICKI JOSEPH M (0001238633) (Reporting),https://www.sec.gov/Archives/edgar/data/123863...
173,"4 - Lumber Liquidators Holdings, Inc. (0001396...",https://www.sec.gov/Archives/edgar/data/139603...
172,4 - BRAINERD MARY K (0001195918) (Reporting),https://www.sec.gov/Archives/edgar/data/119591...
...,...,...
4,424B2 - CITIGROUP INC (0000831001) (Filer),https://www.sec.gov/Archives/edgar/data/831001...
3,13F-HR - BRUNI J V & CO /CO (0001078841) (Filer),https://www.sec.gov/Archives/edgar/data/107884...
2,8-K - Federal Home Loan Bank of Atlanta (00013...,https://www.sec.gov/Archives/edgar/data/133146...
1,6-K - EMX Royalty Corp (0001285786) (Filer),https://www.sec.gov/Archives/edgar/data/128578...
